In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from datetime import datetime, date

In [2]:
# Loading the Customer Address Data from the excel file

cust_address = pd.read_excel("F://Kiran Data//PYTHON//Raw_data.xlsx", sheet_name = 'CustomerAddress')

In [3]:
# Checking first 5 records from Customer Address Data

cust_address.head()

,customer_id,address,postcode,state,country,property_valuation
0,1,060 Morning Avenue,2016,New South Wales,Australia,10
1,2,6 Meadow Vale Court,2153,New South Wales,Australia,10
2,4,0 Holy Cross Court,4211,QLD,Australia,9
3,5,17979 Del Mar Point,2448,New South Wales,Australia,4
4,6,9 Oakridge Court,3216,VIC,Australia,9


In [4]:
# Information of columns and data-types of Customer Address Data.

cust_address.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3999 entries, 0 to 3998
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   customer_id         3999 non-null   int64 
 1   address             3999 non-null   object
 2   postcode            3999 non-null   int64 
 3   state               3999 non-null   object
 4   country             3999 non-null   object
 5   property_valuation  3999 non-null   int64 
dtypes: int64(3), object(3)
memory usage: 187.6+ KB


The data-type of columns looks fine. Let's check for the data quality and apply data cleaning process where ever applicable to clean our dataset before performing any analysis.

## Total Records

In [5]:
# Let's check shape of data

print("Total records (rows) in the dataset : {}".format(cust_address.shape[0]))
print("Total columns (features) in the dataset : {}".format(cust_address.shape[1]))

Total records (rows) in the dataset : 3999
Total columns (features) in the dataset : 6


## Numeric and Non numeric column

In [6]:
# Select all numeric columns
df_numeric = cust_address.select_dtypes(include = [np.number])
numeric_cols = df_numeric.columns.values
print(f"The numeric columns are : {numeric_cols}")

#Select all Non-numeric columns
df_non_numeric = cust_address.select_dtypes(exclude = [np.number])
Non_numeric_cols = df_non_numeric.columns.values
print(f"The Non numeric columns are : {Non_numeric_cols}")


The numeric columns are : ['customer_id' 'postcode' 'property_valuation']
The Non numeric columns are : ['address' 'state' 'country']


## 1. Check Missing Values

Checking for the presence of any missing values in the dataset. If missing values are present for a particular feature then depending upon the situation the feature may be either dropped (cases when a major amount of data is missing) or an appropiate value will be imputed in the feature column with missing values.

In [7]:
# Total number of missing values

cust_address.isnull().sum()

customer_id           0
address               0
postcode              0
state                 0
country               0
property_valuation    0
dtype: int64

There are no missing values in this dataset.

## 2. Check inconsistency in the data

We will check whether there is inconsistent data / typo error data is present in the categorical columns.<br>
The columns to be checked are <b>'address', 'postcode' ,'state', 'country'</b>.

### 2.1 State

In [8]:
cust_address['state'].value_counts()

NSW                2054
VIC                 939
QLD                 838
New South Wales      86
Victoria             82
Name: state, dtype: int64

Here there are inconsistent data in State column. For New South Wales and Victoria we have two values, one being the full name and the other being their short name. The State names should be standardised and columns with state as <b>New South Wales will be replaced by NSW</b> and columns with state as <b>Victoria will be replaced by VIC</b>.

In [9]:
# Defined function to replace full name of state into short name

def replace_state_names(state_name):
    if state_name == 'New South Wales':
        return 'NSW'
    elif state_name == 'Victoria':
        return 'VIC'
    else:
        return state_name
        
# Applying the above fuction to state column
cust_address['state'] = cust_address['state'].apply(replace_state_names)

In [10]:
cust_address['state'].value_counts()

NSW    2140
VIC    1021
QLD     838
Name: state, dtype: int64

After applying the above function the state name is standardised and there is no inconsistency in the state column.

### 2.2 Country

In [11]:
cust_address['country'].value_counts()

Australia    3999
Name: country, dtype: int64

There is no inconsistent data in the country column

### 2.3 Postcode

In [12]:
cust_address[['address','postcode', 'state' , 'country']].drop_duplicates()

,address,postcode,state,country
0,060 Morning Avenue,2016,NSW,Australia
1,6 Meadow Vale Court,2153,NSW,Australia
2,0 Holy Cross Court,4211,QLD,Australia
3,17979 Del Mar Point,2448,NSW,Australia
4,9 Oakridge Court,3216,VIC,Australia
...,...,...,...,...
3994,1482 Hauk Trail,3064,VIC,Australia
3995,57042 Village Green Point,4511,QLD,Australia
3996,87 Crescent Oaks Alley,2756,NSW,Australia
3997,8194 Lien Street,4032,QLD,Australia


The Postcode column looks perfect. There is no inconsistency / typo in the data.

## 3. Check Duplicate records

We need to ensure that there is no duplication of records in the dataset. This may lead to error in data analysis due to poor data quality. 
If there are duplicate rows of data then we need to drop such records.
For checking for duplicate records we need to firstly remove the primary key column of the dataset then apply drop_duplicates() function provided by Python.

In [13]:
# Dropping the primary key column i.e customer_id and storing into a temporary dataframe.

cust_address_no_duplicates = cust_address.drop('customer_id', axis=1).drop_duplicates()

print(f"Number of records after removing customer_id (pk), duplicates : {cust_address_no_duplicates.shape[0]}")
print(f"Number of records in original dataset : {cust_address.shape[0]}")

Number of records after removing customer_id (pk), duplicates : 3999
Number of records in original dataset : 3999


since both numbers are same. so we can say that there are no dupicate records in this dataset.

## 4. Exporting cleaned customer address data to csv

In [14]:
cust_address.to_csv('CustomerAddress_cleaned.csv', index= False)